In [108]:
from datetime import datetime, timedelta
import json
import matplotlib.pyplot as plt
import pandas as pd
import time
import tweepy

In [111]:

auth = tweepy.OAuthHandler(consumer_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                           consumer_secret = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
auth.set_access_token("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
                      "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

api = tweepy.API(auth)

current_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
cutoff_time = (datetime.utcnow() - timedelta(days = 1)).strftime("%Y-%m-%d %H:%M:%S")

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

John Jay, America's first Chief Justice and an author of the Federalist Papers, died #onthisday 1829… https://t.co/3thqsyI1DX
How Trump sharing intel with Russia could create an 'intelligence vacuum' and put Canadians at risk… https://t.co/ul58BnMecp
Pres. Trump is rolling back Obama-era controls over drones; strikes now more indiscriminate, conveniently deniable:… https://t.co/GWyCseEzsZ
Could Donald Trump really get fired from the Presidency? (Yes): https://t.co/DQsgXBCiqG https://t.co/ZBJ1x56qNG
You can fail at what you don’t want, so you might as well take a chance on doing what you love.
JIM CARREY… https://t.co/5EjpGX2em5
RT @Thom_astro: A #timelapse flight over a cloudy part of the world, spot the @Space_Station radiators top-right? https://t.co/PGMCJICJtz h…
RT @toppjj: @tavissmiley @KillerMike @KillerMike killing it on Tavis Smiley #BlackLivesMatter
RT @jackhammer3853: @KillerMike OK Mike! If U Start Shooting Clubs I Will Join!!
Salutes u sir https://t.co/ybKVrQIUBP
West coast

In [3]:
# Set the users of interest.
users = ['nytimes',
        'thesun',
        'thetimes',
        'ap',
        'cnn',
        'bbcnews',
        'cnet',
        'msnuk',
        'telegraph',
        'usatoday',
        'wsj',
        'washingtonpost',
        'bostonglobe',
        'newscomauhq',
        'skynews',
        'sfgate',
        'ajenglish',
        'independent',
        'guardian',
        'latimes',
        'reutersagency',
        'abc',
        'business',
        'bw',
        'time']

In [4]:
tweets_data = []

# For each user, get the most recent tweets and store them in a dataframe.
for user in range(len(users)):
    user = api.get_user(users[user])
    recent_tweets = api.user_timeline(user_id = user.id)
    for tweet in range(len(recent_tweets)):
        json_str = json.dumps(recent_tweets[tweet]._json)
        tweet = json.loads(json_str)
        tweets_data.append(tweet)

In [131]:
tweets = pd.DataFrame()

# Function to get the URL of a tweet even if it's a retweet.
def get_url(tweet):
    return("https://twitter.com/" + tweet['user']['screen_name'] + "/status/" + tweet['id_str'])

tweets['created_at'] = list(map(lambda tweet: tweet['created_at'], tweets_data))
tweets['utc_offset'] = list(map(lambda tweet: tweet['user']['utc_offset'], tweets_data))
tweets['text'] = list(map(lambda tweet: tweet['text'], tweets_data))
tweets['url'] = list(map(get_url, tweets_data))
tweets['screen_name'] = list(map(lambda tweet: tweet['user']['screen_name'], tweets_data))
tweets['name'] = list(map(lambda tweet: tweet['user']['name'], tweets_data))
tweets['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweets_data))
tweets['favorite_count'] = list(map(lambda tweet: tweet['favorite_count'], tweets_data))

# Function to convert the time of the tweets to a proper datetime.
def convert_time(tweet):
    newtime = datetime.strptime(tweets['created_at'][tweet], '%a %b %d %H:%M:%S +0000 %Y') + timedelta(seconds = int(tweets['utc_offset'][tweet]))
    return(newtime)

tweets['utc_created_at'] = list(map(convert_time, range(len(tweets))))

In [133]:
tweets

,created_at,utc_offset,text,url,screen_name,name,retweet_count,favorite_count,utc_created_at
0,Wed May 17 05:49:01 +0000 2017,-14400,Israel’s concerns about Trump’s handling of cl...,https://twitter.com/nytimes/status/86471938596...,nytimes,The New York Times,98,148,2017-05-17 01:49:01
1,Wed May 17 05:32:38 +0000 2017,-14400,The events that led up to James Comey’s firing...,https://twitter.com/nytimes/status/86471525894...,nytimes,The New York Times,221,320,2017-05-17 01:32:38
2,Wed May 17 05:16:04 +0000 2017,-14400,RT @nytimesworld: Trump's acknowledgment that ...,https://twitter.com/nytimes/status/86471109294...,nytimes,The New York Times,90,0,2017-05-17 01:16:04
3,Wed May 17 05:01:03 +0000 2017,-14400,India allows abortion for a 10-year-old girl w...,https://twitter.com/nytimes/status/86470731206...,nytimes,The New York Times,147,406,2017-05-17 01:01:03
4,Wed May 17 04:46:06 +0000 2017,-14400,"Kellyanne Conway takes a ""moment from more imp...",https://twitter.com/nytimes/status/86470354869...,nytimes,The New York Times,52,166,2017-05-17 00:46:06
5,Wed May 17 04:39:15 +0000 2017,-14400,"Villaggio Coppola Journal: On Italy’s Coast, a...",https://twitter.com/nytimes/status/86470182628...,nytimes,The New York Times,60,140,2017-05-17 00:39:15
6,Wed May 17 04:31:03 +0000 2017,-14400,Thailand warned Facebook after a video seemed ...,https://twitter.com/nytimes/status/86469976415...,nytimes,The New York Times,122,177,2017-05-17 00:31:03
7,Wed May 17 04:21:04 +0000 2017,-14400,"""Pay Trump Bribes Here"": An artist projected t...",https://twitter.com/nytimes/status/86469724978...,nytimes,The New York Times,579,1679,2017-05-17 00:21:04
8,Wed May 17 04:15:13 +0000 2017,-14400,"At Cannes, a Rich History of Capturing Politic...",https://twitter.com/nytimes/status/86469577916...,nytimes,The New York Times,80,171,2017-05-17 00:15:13
9,Wed May 17 04:11:06 +0000 2017,-14400,"RT @nytopinion: Rather than impeachment, Ross ...",https://twitter.com/nytimes/status/86469474269...,nytimes,The New York Times,136,0,2017-05-17 00:11:06


In [137]:
old_tweets = pd.read_csv("../data/tweets.csv")
tweets = tweets.append(old_tweets)
tweets = tweets.drop_duplicates()
tweets = tweets[tweets['utc_created_at'] < cutoff_time]
tweets.to_csv("../data/old_tweets.csv")

In [ ]:
# old_tweets.to_csv("")
# time_of_most_recent_notification.to_txt("")
# new_notifications.to_csv("")